In [ ]:
import Pkg
Pkg.activate(;temp=true)
Pkg.add("Revise")
import Revise

# you can comment these all and uncomment them one-by-one as you need them
# to avoid loading more packages than you need
pkgs = [
    "AlgebraOfGraphics",
    "Arrow",
    "BioAlignments",
    "BioSequences",
    "BioSymbols",
    "CairoMakie",
    "Clustering",
    "CodecZlib",
    "Colors",
    "ColorSchemes",
    "Conda",
    "CSV",
    "DataFrames",
    "DataStructures",
    "Dates",
    "DelimitedFiles",
    "Distances",
    "Distributions",
    "DocStringExtensions",
    "Downloads",
    "FASTX",
    "FileIO",
    "GenomicAnnotations",
    "GeoMakie",
    "GFF3",
    "GLM",
    "GraphMakie",
    "Graphs",
    "HDF5",
    "HTTP",
    "JLD2",
    "JSON",
    "Karnak",
    "Kmers",
    "LsqFit",
    "Luxor",
    "Makie",
    "MetaGraphs",
    "Mmap",
    "OrderedCollections",
    "Plots",
    "Primes",
    "ProgressMeter",
    "Random",
    "SparseArrays",
    "Statistics",
    "StatsBase",
    "StatsPlots",
    "TopoPlots",
    "uCSV",
    "XAM",
    "XMLDict"
]
Pkg.add(pkgs)
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end

Pkg.develop(path="$(homedir())/workspace/Mycelia")
import Mycelia
Mycelia.system_overview()

In [ ]:
data_directory = joinpath(dirname(pwd()), "data")

In [ ]:
sample_directories = readdir(joinpath(data_directory, "samples"), join=true)

In [ ]:
kraken_dbs = filter(x -> isdir(x), readdir("$(homedir())/workspace/kraken", join=true))

In [ ]:
# # adjust to database size
kraken_db = "/home/cjprybol/workspace/kraken/k2_pluspfp_20231009"
# kraken_db = "/home/cjprybol/workspace/kraken/k2_pluspfp_08gb_20231009"
memory = 256
threads = 32

In [ ]:
# use me to auto-configure sbatch parameterization
# host = first(readlines(`hostname`))
# @assert occursin(r"\.stanford\.edu", host)

In [ ]:
for sample_directory in sample_directories[2:end]
    fastq_files = filter(x -> occursin(r"\.fq\.gz$", x), readdir(sample_directory, join=true))
    sample_ids = unique(replace.(basename.(fastq_files), r"_[12]\.fq\.gz" => ""))
    for sample_id in sample_ids
        trim_galore_directory = joinpath(sample_directory, sample_id * "_trimgalore")
        # kraken_directories = filter(x -> occursin(r"_kraken$", x) && isdir(x), readdir(trim_galore_directory, join=true))
        # @show kraken_directories
        # for kraken_directory in kraken_directories
        #     rm(kraken_directory, recursive=true)
        # end
        forward = first(filter(x -> occursin(r".unmapped.1.fq.gz", x), readdir(trim_galore_directory, join=true)))
        reverse = first(filter(x -> occursin(r".unmapped.2.fq.gz", x), readdir(trim_galore_directory, join=true)))
        # forward = joinpath(trim_galore_directory, sample_id * "_1_val_1.fq.gz")
        # reverse = joinpath(trim_galore_directory, sample_id * "_2_val_2.fq.gz")
        @assert isfile(forward) && isfile(reverse)
        kraken_dir = first(filter(x -> occursin("kraken", x), readdir(trim_galore_directory, join=true)))
        # @show kraken_dir
        kraken_reports = filter(x -> occursin(r"kraken-report\.tsv$", x), readdir(kraken_dir))
        matching_kraken_reports = filter(x -> occursin(basename(kraken_db), x), kraken_reports)
        if isempty(matching_kraken_reports)
            # cmd = `julia $(homedir())/workspace/Mycelia/scripts/classify-short-reads.jl --forward_reads $(forward) --reverse_reads $(reverse) --kraken_db $(kraken_db) --threads $(threads)`
            # run(cmd)
            cmd = "julia $(homedir())/workspace/Mycelia/scripts/classify-short-reads.jl --forward_reads $(forward) --reverse_reads $(reverse) --kraken_db $(kraken_db) --threads $(threads)"
            submission = 
            `sbatch 
            --mail-user=cameron.prybol@gmail.com
            --mail-type=ALL
            --error=%x-%j.err
            --output=%x-%j.out
            --partition=batch
            --account=mpsnyder
            --nodes=1
            --ntasks=1
            --time=1-00:00:00    
            --cpus-per-task=$(threads)
            --mem=$(memory)G
            --job-name=classify-short-reads
            --wrap $cmd
            `
            run(submission)
        end
    end
end